# Tests

In [1]:
Threads.nthreads()

20

In [2]:
using LinearAlgebra
using Turing
using LimberJack
using NPZ
using ForwardDiff
using BenchmarkTools;
using Interpolations

In [3]:
using PythonCall
np = pyimport("numpy");

In [4]:
meta = np.load("../../data/FD/FD_meta.npz")
names = pyconvert(Vector{String}, meta["names"])
types = pyconvert(Vector{String}, meta["types"])
pairs = pyconvert(Vector{Vector{String}}, meta["pairs"])
idx = pyconvert(Vector{Int}, meta["idx"])
data_vector = pyconvert(Vector{Float64}, meta["cls"])
cov_tot = pyconvert(Matrix{Float64}, meta["cov"])
inv_cov_tot = pyconvert(Matrix{Float64}, meta["inv_cov"]);

In [5]:
print(length(data_vector))

1275

In [6]:
files = npzread("../../data/FD/FD_files.npz");

In [7]:
path = "/home/jaimerz/PhD/LimberJack.jl/chains/carlos_chains/cl_cross_corr_v3_DES_K1000_all_mag_correctMag/"
pars = np.loadtxt(string(path, "cl_cross_corr_v3_DES_K1000_all_mag_correctMag.bestfit"))
pars = pyconvert(Vector{Float64}, pars);


In [8]:
cls_carlos_ND = np.load(string(path, "cl_cross_corr_bestfit_info.npz"))
cls_carlos_SD = np.load(string(path, "cl_cross_corr_bestfit_info_copy.npz"))
cls_carlos_FD = np.append(cls_carlos_SD["cls"], cls_carlos_ND["cls"])
cls_carlos_FD = pyconvert(Vector{Float64}, cls_carlos_FD)
carlos_chi = pyconvert(Float64, - (cls_carlos_ND["chi2"] +  cls_carlos_SD["chi2"]));

## Timings

In [19]:
parss = [pars[1]+pars[2], pars[46], pars[1], pars[5], pars[4],
         pars[6], pars[7], pars[8], pars[9], pars[10], 
         pars[11], pars[12], pars[13], pars[14], pars[15], 
         pars[16], pars[17], 
         pars[18], pars[19], pars[20], pars[21],
         pars[22], pars[23], pars[24], pars[25],
         pars[26], pars[27],
         pars[28], pars[29], pars[30], pars[31],
         pars[32], pars[33], pars[34], pars[35],
         pars[36], pars[37], pars[38], pars[39], pars[40],
         pars[41], pars[42], pars[43], pars[44], pars[45]]

println(parss)

[0.29292871000000004, 0.7881468, 0.04007301, 0.7817957, 0.9117439, 1.477675, 1.805091, 1.783029, 2.173222, 2.215141, -0.005197634, -0.008097385, -0.0008775838, 0.001324318, -0.004479689, 0.272602, -2.410019, 0.04939203, 0.02605237, 0.0261737, -0.007550238, -0.01824749, 0.001053441, 0.004397685, 0.01410571, 2.444282, 2.630131, 1.166097, 1.398773, 1.349232, 1.822822, -0.006161913, 0.006878312, 0.003410951, -0.005914051, -0.005803008, 0.007675705, 0.0044067, -0.008634658, 0.001940326, 0.001886847, -0.005373902, 0.02993692, 0.01237547, -0.0166776]


In [20]:
nuisances = Dict("DESgc__0_b" => pars[6],
                 "DESgc__1_b" => pars[7],
                 "DESgc__2_b" => pars[8],
                 "DESgc__3_b" => pars[9],
                 "DESgc__4_b" => pars[10],
                 "DESgc__0_dz" => pars[11],
                 "DESgc__1_dz" => pars[12],
                 "DESgc__2_dz" => pars[13],
                 "DESgc__3_dz" => pars[14],
                 "DESgc__4_dz" => pars[15],

                 "A_IA" => pars[16],
                 "alpha_IA" => pars[17],

                 "DESwl__0_m" => pars[18],
                 "DESwl__1_m" => pars[19],
                 "DESwl__2_m" => pars[20],
                 "DESwl__3_m" => pars[21],
                 "DESwl__0_dz" => pars[22],
                 "DESwl__1_dz" => pars[23],
                 "DESwl__2_dz" => pars[24],
                 "DESwl__3_dz" => pars[25],

                 "eBOSS__0_b" => pars[26],
                 "eBOSS__1_b" => pars[27],

                 "DECALS__0_b" => pars[28],
                 "DECALS__1_b" => pars[29],
                 "DECALS__2_b" => pars[30],
                 "DECALS__3_b" => pars[31],
                 "DECALS__0_dz" => pars[32],
                 "DECALS__1_dz" => pars[33],
                 "DECALS__2_dz" => pars[34],
                 "DECALS__3_dz" => pars[35],

                 "KiDS1000__0_m" => pars[36],
                 "KiDS1000__1_m" => pars[37],
                 "KiDS1000__2_m" => pars[38],
                 "KiDS1000__3_m" => pars[39],
                 "KiDS1000__4_m" => pars[40],
                 "KiDS1000__0_dz" => pars[41],
                 "KiDS1000__1_dz" => pars[42],
                 "KiDS1000__2_dz" => pars[43],
                 "KiDS1000__3_dz" => pars[44],
                 "KiDS1000__4_dz" => pars[45]);

In [48]:
nuisances["eBOSS__1_b"]

2.630131

In [10]:
function diff_Xi2(pars; data_vector=data_vector, inv_cov=inv_cov_tot)
    nuisances = Dict("DESgc__0_0_b" => pars[6],
                     "DESgc__1_0_b" => pars[7],
                     "DESgc__2_0_b" => pars[8],
                     "DESgc__3_0_b" => pars[9],
                     "DESgc__4_0_b" => pars[10],
                     "DESgc__0_0_dz" => pars[11],
                     "DESgc__1_0_dz" => pars[12],
                     "DESgc__2_0_dz" => pars[13],
                     "DESgc__3_0_dz" => pars[14],
                     "DESgc__4_0_dz" => pars[15],
        
                     "A_IA" => pars[16],
                     "alpha_IA" => pars[17],
        
                     "DESwl__0_e_m" => pars[18],
                     "DESwl__1_e_m" => pars[19],
                     "DESwl__2_e_m" => pars[20],
                     "DESwl__3_e_m" => pars[21],
                     "DESwl__0_e_dz" => pars[22],
                     "DESwl__1_e_dz" => pars[23],
                     "DESwl__2_e_dz" => pars[24],
                     "DESwl__3_e_dz" => pars[25],
        
                     "eBOSS__0_0_b" => pars[26],
                     "eBOSS__1_0_b" => pars[27],

                     "DECALS__0_0_b" => pars[28],
                     "DECALS__1_0_b" => pars[29],
                     "DECALS__2_0_b" => pars[30],
                     "DECALS__3_0_b" => pars[31],
                     "DECALS__0_0_dz" => pars[32],
                     "DECALS__1_0_dz" => pars[33],
                     "DECALS__2_0_dz" => pars[34],
                     "DECALS__3_0_dz" => pars[35],
                    
                     "KiDS1000__0_e_m" => pars[36],
                     "KiDS1000__1_e_m" => pars[37],
                     "KiDS1000__2_e_m" => pars[38],
                     "KiDS1000__3_e_m" => pars[39],
                     "KiDS1000__4_e_m" => pars[40],
                     "KiDS1000__0_e_dz" => pars[41],
                     "KiDS1000__1_e_dz" => pars[42],
                     "KiDS1000__2_e_dz" => pars[43],
                     "KiDS1000__3_e_dz" => pars[44],
                     "KiDS1000__4_e_dz" => pars[45])
    
    Ωm = pars[1]
    s8 = pars[2]
    Ωb = pars[3]
    h = pars[4]
    ns = pars[5]
    
    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, ns, s8, 
                                     tk_mode="emulator",
                                     Pk_mode="Halofit", 
                                     emul_path= "../../emulator/files.npz")
    
    theory = Theory(cosmology, names, types, pairs,
                    idx, files; Nuisances=nuisances)
    diff = @.(theory-data_vector)
    return dot(diff, inv_cov_tot  * diff)

end

diff_Xi2 (generic function with 1 method)

In [11]:
function fd(pars)
    n = length(pars)
    grad  = zeros(length(pars))
    for i in 1:n
        d = zeros(n)
        e = 0.005
        d[i] +=  e
        grad[i] = (diff_Xi2(pars .+ d) .- diff_Xi2(pars .- d)) ./ (2*e)
    end
    return grad
end

fd (generic function with 1 method)

In [12]:
@benchmark fd(parss)

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  3.110 s …  3.120 s  ┊ GC (min … max): 8.49% … 8.52%
 Time  (median):     3.115 s             ┊ GC (median):    8.50%
 Time  (mean ± σ):   3.115 s ± 7.472 ms  ┊ GC (mean ± σ):  8.50% ± 0.02%

  █                                                      █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  3.11 s        Histogram: frequency by time        3.12 s <

 Memory estimate: 6.44 GiB, allocs estimate: 18099884.

In [13]:
@benchmark ForwardDiff.gradient(diff_Xi2, parss)

BenchmarkTools.Trial: 16 samples with 1 evaluation.
 Range (min … max):  296.988 ms … 362.935 ms  ┊ GC (min … max): 13.79% … 15.06%
 Time  (median):     318.265 ms               ┊ GC (median):    13.22%
 Time  (mean ± σ):   322.900 ms ±  17.477 ms  ┊ GC (mean ± σ):  13.46% ±  0.90%

                █    █      ▃                                    
  ▇▁▁▁▁▁▁▁▁▁▇▁▁▇█▁▁▁▁█▁▇▁▁▁▁█▁▁▇▁▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇▇ ▁
  297 ms           Histogram: frequency by time          363 ms <

 Memory estimate: 2.19 GiB, allocs estimate: 870894.

In [14]:
@benchmark diff_Xi2(parss)

BenchmarkTools.Trial: 147 samples with 1 evaluation.
 Range (min … max):  24.288 ms … 63.371 ms  ┊ GC (min … max): 0.00% … 28.78%
 Time  (median):     28.149 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   34.161 ms ± 10.105 ms  ┊ GC (mean ± σ):  9.70% ± 14.17%

    ▆█▇▁                                                       
  ▃▄████▅▆▃▁▃▁▁▃▃▃▅▄▁▃▃▃▄▅▁▃▃▅▃▁▃▃▃▄▃▃▃▁▃▃▁▁▄▁▄▁▁▁▃▁▁▃▃▁▁▃▃▁▃ ▃
  24.3 ms         Histogram: frequency by time        62.1 ms <

 Memory estimate: 73.24 MiB, allocs estimate: 201103.

In [ ]:
npzwrite(string(, ".npz"), [evall, diff])

In [16]:
using ReverseDiff
ftape_diff_Xi2 = ReverseDiff.GradientTape(diff_Xi2, pars)
compiled_ftape_diff_Xi2 = ReverseDiff.compile(ftape_diff_Xi2)

typename(ReverseDiff.CompiledTape)(diff_Xi2)

In [17]:
Xi2 = similar(pars);

In [18]:
@benchmark ReverseDiff.gradient!(Xi2, compiled_ftape_diff_Xi2, pars)

LoadError: InterruptException:

In [15]:
@benchmark ForwardDiff.jacobian(diff_cls, pars)

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  2.305 s …   2.404 s  ┊ GC (min … max): 8.40% … 8.07%
 Time  (median):     2.340 s              ┊ GC (median):    8.29%
 Time  (mean ± σ):   2.350 s ± 50.542 ms  ┊ GC (mean ± σ):  8.35% ± 0.26%

  █                   █                                   █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.3 s          Histogram: frequency by time         2.4 s <

 Memory estimate: 3.76 GiB, allocs estimate: 1122396.

In [ ]:
@benchmark ReverseDiff.jacobian(diff_cls, pars)

## Xi2

In [43]:
best_pred = make_cls(pars)

610-element Vector{Float64}:
 3.729976949631498e-5
 1.924863264546964e-5
 1.2198839863768642e-5
 2.7338998158421804e-5
 1.8302492387509104e-5
 1.1697676470225297e-5
 8.461577277803604e-6
 6.2847518106000625e-6
 4.953759872071052e-6
 4.0277958603799074e-6
 2.0247593758184622e-5
 1.3879505201281076e-5
 1.0208564507478019e-5
 ⋮
 1.481014723350936e-9
 1.2597256449462859e-9
 1.073704667701235e-9
 9.174815729181471e-10
 7.858916381457959e-10
 6.74149817464629e-10
 5.784141317368861e-10
 4.954936101687788e-10
 4.234671163674685e-10
 3.6091137647921583e-10
 3.0662205257958187e-10
 2.5945231667605537e-10

In [44]:
best_Xi = dot((best_pred .- data_vector), inv_cov_tot  * (best_pred .- data_vector))

726.9778969921368

In [45]:
reduced_best_Xi = round(best_Xi/(length(data_vector)-43), digits=3)

1.282

## Prior sampling

In [21]:
N_samples = 100
cls_preds = zeros(N_samples, length(data_vector))
for i in 1:N_samples
    Ωm = rand(Uniform(0.1, 0.6))
    s8 = rand(Uniform(0.6, 0.9))
    cls_preds[i,:] = make_cls(pars; Ωm=Ωm, s8=s8)
end

In [22]:
cls_preds_m, cls_preds_s = mean(cls_preds, dims=1), std(cls_preds, dims=1);

In [23]:
npzwrite("jaime_cls.npz", Dict("cls_best" => best_pred,
                               "cls_m" => cls_preds_m,
                               "cls_s" => cls_preds_s))

## Turing

In [9]:
@model function model(data_vector::Vector{Float64})
    Ωm ~ Uniform(0.1, 0.6)
    Ωb = 0.05 #~ Uniform(0.03, 0.07)
    h = 0.67 #~ Uniform(0.6, 0.9)
    s8 = 0.81 #~ Uniform(0.6, 1.0)
    ns = 0.96 #~ Uniform(0.87, 1.07)

    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, s8, s8,
                                     tk_mode="emulator",
                                     Pk_mode="Halofit")
    theory = Theory(cosmology,
                    tracers_names, pairs,
                    idx, files)
    data_vector ~ MvNormal(theory, cov_tot)
    return data_vector
end;

In [10]:
iterations = 300
nadapts = 100;

In [11]:
chain3 = sample(model(data_vector), NUTS(5, 0.1; init_ϵ=0.03), iterations,
                progress=true; save_state=true);

┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/iWHPQ/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/iWHPQ/src/hamiltonian.jl:47
Sampling:   1%|▍                                        |  ETA: 0:07:02┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/iWHPQ/src/hamiltonian.jl:47
Sampling: 100%|█████████████████████████████████████████| Time: 0:01:28


LoadError: InterruptException:

In [12]:
chain1 = sample(model(data_vector), MH(), iterations,
            nadapt=nadapts, progress=true; save_state=true);

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:24


LoadError: InterruptException:

In [4]:
files = npzread("../data/FD/K1K_DELS_DESY1_eBOSS_CMBk_files.npz")
products = npzread("products.npz")

Dict{String, Array} with 5 entries:
  "idx"         => [0, 5, 13, 23, 34, 47, 52, 57, 62, 67  …  1077, 1099, 1121, …
  "cls_m"       => [0.000229969 7.55142e-5 … 3.71104e-10 3.14893e-10]
  "err"         => [5.919e-5, 1.57008e-5, 6.64192e-6, 3.82552e-6, 2.56294e-6, 2…
  "cls_s"       => [0.000216359 3.16836e-5 … 2.44088e-10 2.0759e-10]
  "data_vector" => [0.000190242, 7.39128e-5, 4.26125e-5, 2.22966e-5, 2.17688e-5…

In [3]:
figure, axis = plt.subplots(5, figsize=(5,14))
l = 1
for i in 0:4
    axis[i+1].errorbar(files[string("ls_1", i, "1", i)], 
                     data[idx[l]:idx[l+1]-1], 
                     yerr=err[idx[l]:idx[l+1]-1], fmt="gs", label="Data")
    axis[i+1].plot(files[string("ls_1", i, "1", i)],
                   cls_preds_m[idx[l]:idx[l+1]-1],
                   "bo-", label="Preds")
    axis[i+1].fill_between(files[string("ls_1", i, "1", i)],
                   cls_preds_m[idx[l]:idx[l+1]-1].+cls_preds_s[idx[l]:idx[l+1]-1],
                   cls_preds_m[idx[l]:idx[l+1]-1].-cls_preds_s[idx[l]:idx[l+1]-1],
                   color="b", alpha=.5)
    axis[i+1].set_title(string("GC", i, "GC", i))
    l += 1
end
plt.legend()
plt.show()
print(l)

LoadError: UndefVarError: files not defined